# Additional Functions

### Objectives
1. Apply built-in functions to generate data for new columns
2. Apply DataFrame NA functions to handle null values
3. Join DataFrames

### Methods
- [DataFrame Methods](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.join.html): `join`
- [DataFrameNaFunctions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameNaFunctions.html): `fill`, `drop`
- [Built-In Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html):
  - Aggregate: `collect_set`
  - Collection: `explode`
  - Misc: `col`, `lit`


### Generate inline JSON sales data and load it

In [ ]:
import pandas as pd

sales_data = [
    {"email": "a@gmail.com", "items": [{"item_name": "Mattress", "coupon": None}]},
    {"email": "b@yahoo.com", "items": [{"item_name": "Pillow", "coupon": "DISCOUNT10"}]},
    {"email": "c@gmail.com", "items": [{"item_name": "Blanket", "coupon": None}]}
]

pdf = pd.DataFrame(sales_data)
sales_path = "/dbfs/tmp/sales_data.json"
pdf.to_json(sales_path, orient="records", lines=True)

sales_df = spark.read.option("multiline", "false").json(sales_path)
display(sales_df)

### Non-aggregate and Miscellaneous Functions

In [ ]:
from pyspark.sql.functions import col, lit

gmail_accounts = sales_df.filter(col("email").endswith("gmail.com"))
display(gmail_accounts)

In [ ]:
display(gmail_accounts.select("email", lit(True).alias("gmail user")))

### Handling Nulls with DataFrameNaFunctions

In [ ]:
print(sales_df.count())
print(sales_df.na.drop().count())

In [ ]:
from pyspark.sql.functions import explode

sales_exploded_df = sales_df.withColumn("items", explode(col("items")))
display(sales_exploded_df.select("items.coupon"))
print(sales_exploded_df.select("items.coupon").count())
print(sales_exploded_df.select("items.coupon").na.drop().count())

In [ ]:
display(sales_exploded_df.select("items.coupon").na.fill("NO COUPON"))

### Joining DataFrames

In [ ]:
users_data = [
    {"email": "a@gmail.com", "user_id": 1},
    {"email": "b@yahoo.com", "user_id": 2},
    {"email": "c@gmail.com", "user_id": 3}
]

users_pdf = pd.DataFrame(users_data)
users_path = "/dbfs/tmp/users_data.json"
users_pdf.to_json(users_path, orient="records", lines=True)

users_df = spark.read.option("multiline", "false").json(users_path)
display(users_df)

In [ ]:
joined_df = gmail_accounts.join(other=users_df, on='email', how="inner")
display(joined_df)